In [1]:
# Import libraries
import pandas as pd
import numpy as np
import random
import math

In [2]:
# Declare variables
# iterations (n)
n = 1_000_000

# winning odds (r)
r = 0.49

# base bet (b)
b = 10

# table maximum (M)
m = 1000

In [3]:
# Calculate max consecutive losses before table max is hit (mcl)

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

mcl = round_down(math.log(m/b,2))+2
print(mcl)

8.0


In [4]:
# Generate win/loss array
win_loss_array = []
for i in range (0,n):
    win_loss_generation = random.choices([0,1], weights = [1-r,r])
    win_loss_number = win_loss_generation[0]
    if win_loss_number >= 0.5:
        win_loss_array += 'W'
    else:
        win_loss_array += 'L'

# Generate dummy array for the simulated data
consecutive_l = [None] * n
bet_amount = [None] * n
round_cashflow = [None] * n
max_bet_counter = [None] * n

# Print first 10 items of the resultant arrays
print('The first 10 items of the win/loss array:')
print(win_loss_array[:10])

The first 10 items of the win/loss array:
['W', 'L', 'L', 'L', 'W', 'L', 'W', 'L', 'W', 'W']


In [5]:
# Generate base dataframe
df = pd.DataFrame({'W/L' : win_loss_array[:],
                   'Consecutive L': consecutive_l[:],
                   'Max Bet Counter': max_bet_counter[:],
                   'Bet Amount': bet_amount[:],
                   'Round Cashflow': round_cashflow[:],
                  })
print('The first 5 items of the resultant dataframe:')
display(df.head())

The first 5 items of the resultant dataframe:


,W/L,Consecutive L,Max Bet Counter,Bet Amount,Round Cashflow
0,W,None,None,None,None
1,L,None,None,None,None
2,L,None,None,None,None
3,L,None,None,None,None
4,W,None,None,None,None


In [6]:
# Count consecutive L
if df.iloc[0,0] == 'W':
    df.iloc[0,1] = 0
else:
    df.iloc[0,1] = 1
    
for i in range(1,n):
    if df.iloc[i,0] == 'L':
        df.iloc[i,1] = df.iloc[i-1,1]+1
    else:
        df.iloc[i,1] = 0

In [7]:
# Generate max bet counter
df.iloc[0,2] = 0

for i in range(1,n):
    # Scenario 1: max bet counter is 0 and max consecutive loss is not reached
    if df.iloc[i,1] < mcl and df.iloc[i-1,2] == 0:
        df.iloc[i,2] = 0
    # Scenario 2: max bet counter is 0 and we lost the first max table bet
    elif df.iloc[i,1] == mcl and df.iloc[i-1,2] == 0 and df.iloc[i,0] == 'L':
        df.iloc[i,2] = 2
    # Scenario 3: max bet counter has started counting and we l lose
    elif df.iloc[i-1,2] > 0 and df.iloc[i,0] == 'L':
        df.iloc[i,2] = df.iloc[i-1,2]+1
    # Scenario 4: max bet counter has started counting and we win
    elif  df.iloc[i-1,2] > 0 and df.iloc[i,0] == 'W':
        df.iloc[i,2] = df.iloc[i-1,2]-1

In [8]:
# Generate bet amount

# Start initial bet (b)
df.iloc[0,3] = b

# Second bet is always (b) regardless of the outcome of the first round
df.iloc[1,3] = b

for i in range(2,n):
    # Scenario 1: max bet not hit, last 2 games were losses
    if df.iloc[i,2] == 0 and df.iloc[i-1,0] == 'L' and df.iloc[i-2,0] == 'L':
        df.iloc[i,3] = df.iloc[i-1,3]*2
    # Scenario 2: max bet not hit, won last game
    elif df.iloc[i,2] == 0 and df.iloc[i-1,0] == 'W':
        df.iloc[i,3] = b
    # Scenario 3: max bet not hit, previous round lost but the one before is win
    elif df.iloc[i,2] == 0 and df.iloc[i-1,0] == 'L' and df.iloc[i-2,0] == 'W':
        df.iloc[i,3] = b
    # Scenario 4: max bet hit, need to place max bet
    elif df.iloc[i,2] > 0:
        df.iloc[i,3] = (2**(mcl-2))*b

In [9]:
# Generate round cashflow
for i in range(0,n):
    if df.iloc[i,0] == 'L':
        df.iloc[i,4] = df.iloc[i,3]*-1
    else:
        df.iloc[i,4] = df.iloc[i,3]

In [10]:
# Generate total cashflow of previous rounds
df['Total Cashflow'] = df['Round Cashflow'].cumsum()

In [11]:
# check last 5 rows to check for simulation completion
df.iloc[-5:]

,W/L,Consecutive L,Max Bet Counter,Bet Amount,Round Cashflow,Total Cashflow
999995,L,2,19489,640.0,-640.0,-12470730.0
999996,L,3,19490,640.0,-640.0,-12471370.0
999997,W,0,19489,640.0,640.0,-12470730.0
999998,W,0,19488,640.0,640.0,-12470090.0
999999,W,0,19487,640.0,640.0,-12469450.0


In [12]:
# Summary statistics
print('Summary statistics: \n')

print("Total iterations ran: %s" % '{:,}'.format((n)))
print("Total wins: %s" % '{:,}'.format((df['W/L'].value_counts()['W'])))
print("Total losses: %s" % '{:,}'.format((df['W/L'].value_counts()['L'])))
print("Win rate: %s" % "{0:.02%}".format((df['W/L'].value_counts()['W'])/n))
print("Expected Win rate: %s \n" % "{0:.02%}".format(r))

print("Most consecutive losses in a row: %s" % '{:,}'.format(df['Consecutive L'].max()))
print("Lowest Round Cashflow: %s" % '{:,}'.format(df['Round Cashflow'].min()))
print("Lowest Total Cashflow: %s" % '{:,}'.format(df['Total Cashflow'].min()))
print("Final Total Cashflow: %s" % '{:,}'.format(df.iloc[-1,5]))

Summary statistics: 

Total iterations ran: 1,000,000
Total wins: 490,264
Total losses: 509,736
Win rate: 49.03%
Expected Win rate: 49.00% 

Most consecutive losses in a row: 19
Lowest Round Cashflow: -640.0
Lowest Total Cashflow: -12,472,650.0
Final Total Cashflow: -12,469,450.0


In [13]:
# Generate final dataframe for sanity check
df.iloc[:20]

,W/L,Consecutive L,Max Bet Counter,Bet Amount,Round Cashflow,Total Cashflow
0,W,0,0,10,10,10
1,L,1,0,10,-10,0
2,L,2,0,10,-10,-10
3,L,3,0,20,-20,-30
4,W,0,0,40,40,10
5,L,1,0,10,-10,0
6,W,0,0,10,10,10
7,L,1,0,10,-10,0
8,W,0,0,10,10,10
9,W,0,0,10,10,20
